# Measuring the effectiveness Few-shot learning for traffic sign recognition

# ABSTRACT
Nowadays, self-driving veichles is one of the main application of Artificial Intelligence (AI) which has been most widely used in daily-life scenarios. These veichles are based on many AI models among which there are traffic sign recognition systems. Different proposals have been provided for these systems, most of which rely on the usage of big datasets such as GTSRB (dataset of German traffic signs). Collecting a big amount of data is one of the main problems of Machine Learning models, which need lots of evidences in order to being able to generalize well. The effort in collecting more data than available can be one of the main aspects to consider during the development of these systems. Furthermore, using a big dataset expose to possible Adversarial attacks, from poisoning to evasion ones, specially in case of out-source training and testing. For these reasons, in this work we investigate the application of state-of-art Few-shot learning models on the traffic sign recognition domain. The aim of this work is to define the effectiveness of these approaches, in contrast to non-few-shot ones, to open the possibilities of build more robust and effortless solutions in this field.

# NOTE
più peso alla motivazione dell'attacco rispetto alla difficoltà nella raccolta di dati. Quindi focalizzarsi sul problema che attacchi di poisoning necessitano di aggiungere esempi poisonati che sono facilmente scopribili in casi di dataset piccoli (come nel few-shot) mediante un controllo umano dei sample. Attenzione: verificare, in caso questo controllo non si possa fare, se l'attacco è avvantaggiato con un dataset più piccolo rispetto ad uno grande. Se cosi fosse, sarebbe un problema.

in the related work there are the works we are trying to improve

specificare come ho creato il dataset ridotto

testare i modelli anche sul dataset ridotto

cercare come vengono valutati i modelli di few shot. Se si usa bootstrap o altre tecniche

usare più modelli few shot, non solo CLIP e LDC

-  N-way-k-shot (it means N classes and K instances per classes)

-  Approfondire discorso support set (set non utilizzato nè durante training nè durante test ma solo durante fase di predizione)


# RICERCHE ESISTENTI
- <b>FUSED-Net: Enhancing Few-Shot Traffic Sign Detection with Unfrozen Parameters, Pseudo-Support Sets, Embedding Normalization, and Domain Adaptation</b>: https://arxiv.org/html/2409.14852v1 SOLO SU DATASET DEL BANGLADESH

- <b>Cross-domain Few-shot In-context Learning for Enhancing Traffic Sign Recognition</b>: questo fa un compare tra modelli few shot sul dataset GTSRB

- <b>Few-shot traffic sign recognition with clustering inductive bias and random neural network</b>: questo propone un nuovo modello per few-shot su traffic sign recognition

- <b>Meta-YOLO: Meta-Learning for Few-Shot Traffic Sign Detection via Decoupling Dependencies</b>: questo fa traffic sign detection e non recognition (usa YOLO) quindi a noi non ci interessa

- <b>Research on a Traffic Sign Recognition Method under Small Sample Conditions</b>: nuovo modello proposto

- <b>Self-supervised few-shot learning for real-time traffic sign classification</b>: altro modello introdotto

# EVALUATION METRICS

Possible evaluation metrics used in the papers above are:
Ma precision e recall non sono per binary classification? Lo dovresti fare per ogni classe considerando tutte le altre classi come istanze negative?
- Average precision (AP): area under the precision-recall curve
- Precision
- Recall
- A sto punto anche F1 Score? Male non fa
- Accuracy rate (consigliato al posto di error rate. più utilizzato in letteratura)
-

# EVALUATION TECHNIQUES FOR FEW SHOTS ALGORITHMS
- <b>Bootstrap</b>
- <b>Fine-Grained Image Classification</b>: si valutano e testano gruppi di classi alla volta in modo tale da identificare i gruppi più problematici (nel nostro caso il gruppo dei segnali stradali relativi ai limiti di velocità potrebbero essere piu difficili da classificare rispetto a quelli di pericolo). "Fine-Grained Image Classification is a task in computer vision where the goal is to classify images into subcategories within a larger category. For example, classifying different species of birds or different types of flowers. This task is considered to be fine-grained because it requires the model to distinguish between subtle differences in visual appearance and patterns, making it more challenging than regular image classification tasks."(https://paperswithcode.com/task/fine-grained-image-classification)
-<b>Generalization error</b>: How much does accuracy drop when we reduce K (e.g., from K=5 to K=1)?*


# EVALUATION TECHNIQUES TO COMPARE THE FEW SHOT VS MANY-SHOT LEARNING
- <b>T-test</b> Separiamo il dataset di test in più dataset dove ciascun dataset contiene i sample di un determinato gruppo di segnali stradali (quindi un gruppo di classi) (vedere Fine-Grained Image Classification in  EVALUATION TECHNIQUES FOR FEW SHOTS ALGORITHMS). Siccome i dataset sono indipendenti gli uni dagli altri possiamo tranquillamente applicare il t-test classico per vedere se l'accuracy rate dell'algoritmo few shot e l'accuracy rate dell'algoritmo non-few shot sono significativamente diversi.





# Nuova sezione

<h2>Code used to create the random dataset:</h2>
import os
import random
import shutil
from PIL import Image

def copy_random_images(p1, p2, num_images_per_folder=5):
    """
    Copies random images from subfolders in p1 to p2, recreating the subfolder structure.

    Args:
        p1 (str): Source path containing subfolders with images.
        p2 (str): Destination path where the subfolder structure and images will be recreated.
        num_images_per_folder (int): Number of random images to copy from each subfolder.
    """
    if not os.path.exists(p1):
        raise ValueError(f"Source path '{p1}' does not exist.")
    
    if not os.path.exists(p2):
        os.makedirs(p2)

    # Iterate through subfolders in p1
    for subfolder in os.listdir(p1):
        subfolder_path = os.path.join(p1, subfolder)
        if os.path.isdir(subfolder_path):
            # Create the same subfolder in p2
            dest_subfolder_path = os.path.join(p2, subfolder)
            os.makedirs(dest_subfolder_path, exist_ok=True)

            # Get all image files in the current subfolder
            image_files = [f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))]
            
            # Select random images
            random_images = random.sample(image_files, min(num_images_per_folder, len(image_files)))

            # Copy selected images to the destination subfolder
            for image in random_images:
                src_image_path = os.path.join(subfolder_path, image)
                dest_image_path = os.path.join(dest_subfolder_path, image)

                # Open, resize, and save the image
                with Image.open(src_image_path) as img:
                    img = img.convert("RGB")  # Ensure the image is in RGB format
                    img = img.resize((32, 32))  # Resize to 32x32
                    img.save(dest_image_path)

            print(f"Copied {len(random_images)} images from '{subfolder_path}' to '{dest_subfolder_path}'.")

p1 = "ortiginal_images_folder"
p2 = "destination_images_folder"
num_images_per_folder = 30

arr=["/train","/val","/test"]

for a in arr:
  copy_random_images(p1, p2, num_images_per_folder+a)

# Setup environment

In [1]:
!git clone https://github.com/mattiacolucci/few-shot-traffic-sign

!pip install ftfy
!pip install alisuretool

Cloning into 'few-shot-traffic-sign'...
remote: Enumerating objects: 3857, done.
remote: Counting objects: 100% (3857/3857), done.
remote: Compressing objects: 100% (3842/3842), done.
remote: Total 3857 (delta 18), reused 3845 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (3857/3857), 10.28 MiB | 27.06 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [7]:
import importlib.util
import sys

# allows to import py files from this folder
sys.path.append("./few-shot-traffic-sign/")

# import train py file
'''
spec = importlib.util.spec_from_file_location("__main__", "./few-shot-traffic-sign/ldc_train.py")
train = importlib.util.module_from_spec(spec)
sys.modules["__main__"] = train
'''

'\nspec = importlib.util.spec_from_file_location("__main__", "./few-shot-traffic-sign/ldc_train.py")\ntrain = importlib.util.module_from_spec(spec)\nsys.modules["__main__"] = train\n'

In [ ]:
from ldc_train import *

dataset_name="artwork"
shots=16
save_dir = dataset_name+"_100_way_"+str(shots)+"_shot_elastic_reg"
log_txt_path = Tools.new_dir(os.path.join(LOG_ROOT, save_dir, "results.txt"))

# Create the configuration for the custom dataset
config = Config10Dataset(
    dataset_name=dataset_name,
    seed=2024,
    shots=shots,
    backbone="ViT-B/16",
    lr=0.001,
    batch_size=64,
    train_epoch=50,
    loss_lambda=[1.0, 1.0, 1.0, 1.0, 1.0, 1.0],   #last is regularization lambda
    fuse_type=2,
    save_dir=save_dir,
    regularization='elastic_net'
)

# Initialize the Runner with the custom dataset configuration
runner = Runner(config=config)

# Start the training process
Tools.print("Starting training with the custom dataset...")
acc_list = runner.train()
Tools.print({"name": "artwork", "acc": acc_list, "detail": config.get_detail()}, log_txt_path)

# Optionally, evaluate the model
Tools.print("Evaluating the model...")
test_acc_list = runner.test()
Tools.print({"name": "artwork", "test_acc": test_acc_list}, log_txt_path)
